In [329]:
import cv2
import numpy as np
import pcl
import json
import freenect
import frame_convert2
import matplotlib.pylab as plt
import matplotlib

import redis
import frame_convert2
import importlib
import time
from scipy.optimize import curve_fit

In [332]:
r = redis.Redis(charset='utf-8', decode_responses=True)
simulation = False
if simulation:
    JOINT_ANGLES_KEY = "sai2::cs225a::panda_robot::sensors::q"
    JOINT_VELOCITIES_KEY = "sai2::cs225a::panda_robot::sensors::dq"
    JOINT_TORQUES_COMMANDED_KEY = "sai2::cs225a::panda_robot::actuators::fgc"
    # 
    CURRENT_POSITION_KEY = "sai2::cs225a::panda_robot::sensors::current_position"
    TARGET_POSITION_KEY = "sai2::cs225a::panda_robot::control::target_position"
else:    
    JOINT_TORQUES_COMMANDED_KEY = "sai2::FrankaPanda::actuators::fgc"
    JOINT_ANGLES_KEY  = "sai2::FrankaPanda::sensors::q"
    JOINT_VELOCITIES_KEY = "sai2::FrankaPanda::sensors::dq"
    JOINT_TORQUES_SENSED_KEY = "sai2::FrankaPanda::sensors::torques"
    MASSMATRIX_KEY = "sai2::FrankaPanda::sensors::model::massmatrix"
    CORIOLIS_KEY = "sai2::FrankaPanda::sensors::model::coriolis"
    ROBOT_GRAVITY_KEY = "sai2::FrankaPanda::sensors::model::robot_gravity"
    # 
    CURRENT_POSITION_KEY = "sai2::FrankaPanda::sensors::current_position"
    TARGET_POSITION_KEY = "sai2::FrankaPanda::control::target_position"
    CALI_DONE_KEY = "sai2::FrankaPanda::control::cali_done"

# Conversion between Redis key-value format to numpy arrays
 
def str_to_vec(s):
    return np.array([float(x) for x in s[1:-1].split(',')])

def key_to_vec(r, key_name):
    return str_to_vec(r.get(key_name))

def vec_to_str(v):
    return '[' + ','.join(x for x in v) + ']'

In [2]:
fx, fy = 529.215, 525.5639
cx, cy = 328.942, 267.480

In [417]:
# Get raw depth and RGB image using libfreenect driver

def get_depth():
    """
    Get depth image in millimeters (valued 0-10000)
    """
    value = freenect.sync_get_depth(format=freenect.DEPTH_REGISTERED)[0]
    return value

def get_video():
    """
    Get BGR frames (valued 0-255)
    """
    return frame_convert2.video_cv(freenect.sync_get_video()[0])

reference_color = np.array([122, 120, 35]) / 255.
hue = matplotlib.colors.rgb_to_hsv(reference_color)[0]
    
def find_color_pixel_uv(rgb):
    hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV) / 255.
    hue_hsv, sat_hsv, bright_hsv = hsv[:, :, 0], hsv[:, :, 1], hsv[:, :, 2]
    
    height, width = hsv.shape[0], hsv.shape[1]
    grid = np.zeros((height, width))    
    grid[(hue_hsv < (hue + 25 / 360.)) & (hue_hsv > (hue - 25 / 360.)) & (sat_hsv > 0.3) & (bright_hsv > 0.3)] = 1
    return np.argwhere(grid > 0)

def to_xyz(rgb, depth):
    z = depth[rgb[:, 0], rgb[:, 1]]
    x = ((rgb[:, 0] - cx) * z) / fx
    y = ((rgb[:, 1] - cy) * z) / fy
    x, y, z = x.reshape(-1, 1), y.reshape(-1, 1), z.reshape(-1, 1)
    xyz = np.hstack([x, y, z])
    xyz = xyz.astype(np.float32)
    return xyz[xyz[:, 2] > 0]


def filter_outlier(pc):
    n_pc = len(pc)
    cloud = pcl.PointCloud()
    cloud_filtered = pcl.PointCloud()
    cloud.from_array(pc)

    kdtree = cloud.make_kdtree_flann()
    [ind, sqdist] = kdtree.nearest_k_search_for_cloud(cloud, int(n_pc / 10))    
    valid_mask = sqdist.max(axis=-1) < 1e-2
    pc_filtered = pc[valid_mask]
    return pc_filtered


def find_ball():
    image = get_video()
    depth = get_depth() / 1000
    
    color_uv = find_color_pixel_uv(image)
    pc_color = to_xyz(color_uv, depth)

    pc_filtered = filter_outlier(pc_color)
    
    if len(pc_filtered) == 0:
        raise Exception("ball not found")
    # ball_center = np.mean(pc_filtered, axis=0)
    r, cx, cy, cz = sphereFit(pc_filtered[:, 0], pc_filtered[:, 1], pc_filtered[:, 2])
    return r[0], cx[0], cy[0], cz[0]

In [472]:
find_ball()

(0.023842133696458604,
 -0.12437976536049923,
 0.42208949529446693,
 2.1410549996398167)

In [465]:
def move_robot(target_xyz, velocity_threshold=1e-1, timeout=5.0):
    r.set(TARGET_POSITION_KEY, vec_to_str(target_xyz))
    start_time = time.time()
    
    while np.norm(key_to_vec(JOINT_VELOCITIES_KEY)) < velocity_threshold:
        curr_time = time.time()
        if curr_time - start_time > timeout:
            raise Exception("control timeout!")

    return key_to_vec(CURRENT_POSITION_KEY)

In [331]:
target_coords = [[0.35, 0.4, 0.3], [0.35, -0.4, 0.3], [0.45, 0.3, 0.8], [0.45, -0.3, 0.8]]

def get_coords(target_coords):
    results_camera = []
    results_robot  = []
    for target_xyz in target_coords:
        try:
            r_robot = move_robot(target_xyz) # tune params! 
            _ = input("Please press a key...")
            r_camera = find_ball()
            results_camera.append(r_camera)
            results_robot.append(r_robot)
        except Exception as e:
            print("{} failed!".format(target_xyz))
            print(e)
    return results_camera, results_robot

In [11]:
def optimize_transformation(xyzs_camera, xyzs_robot):
    """
    args: 
      xyzs_camera = (N, 3) numpy array, containing points in camera frame
      xyzs_robot  = (N, 3) numpy array, containing points in robot frame
    returns:
      T = [4, 4] including a transformation matrix converting camera frame to robot frame
        = [[R(3x3) , t(3x1)],
           [0, 0, 0,      1]]
    """
    centroid_frame_camera = np.mean(xyzs_camera, axis=0)
    centroid_frame_robot  = np.mean(xyzs_robot, axis=0)
    H = (xyzs_camera - centroid_frame_camera).T @ (xyzs_robot - centroid_frame_robot)
    U, S, V = np.linalg.svd(H)
    R = (U @ V).T
    t = centroid_frame_robot - R @ centroid_frame_camera
    return np.hstack([np.vstack([R.T, t]), np.array([[0, 0, 0, 1]]).T]).T

In [375]:
# https://jekel.me/2015/Least-Squares-Sphere-Fit/
def sphereFit(spX,spY,spZ):
    #   Assemble the A matrix
    spX = np.array(spX)
    spY = np.array(spY)
    spZ = np.array(spZ)
    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A, f, rcond=None)

    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = np.sqrt(t)

    return radius, C[0], C[1], C[2]

# Check Transformation Matrix Quality

In [33]:
import numpy as np
from scipy.linalg import lstsq
def check_transformation_matrix(optimize_transformation, num_points, point_scale=5, gaussian_scale=0.3):
    T_1_to_2 = np.array([
        [0.07143, -0.6589, 0.7488, -3], 
        [0.9446, 0.2857, 0.1613, 5],
        [-0.3202, 0.6958, 0.6429, 7], 
        [0, 0, 0, 1]
    ])
    
    points_frame_1 = np.random.rand(num_points, 3) * point_scale
    points_frame_2 = np.pad(points_frame_1, [(0, 0), (0, 1)], 'constant', constant_values=1) @ T_1_to_2.T
    noise = np.random.normal(loc=0.0, scale=gaussian_scale, size=points_frame_2.shape)
    noise[:, 3] = 0
    points_frame_2_noise = points_frame_2 + noise
    
    points_frame_2_ = points_frame_2[:, :3]
    points_frame_2_noise_ = points_frame_2_noise[:, :3]
    
    T_calculated = optimize_transformation(points_frame_1, points_frame_2_noise_)
    points_frame_2_est = np.pad(points_frame_1, [(0, 0), (0, 1)], 'constant', constant_values=1) @ T_calculated.T
    points_frame_2_est = points_frame_2_est[:, :3]
    
    print(f'Original T: \n{T_1_to_2}')
    print(f'Calculated T: \n{T_calculated}')
    print(f'|T[Orig] - T[Calc]| = {np.linalg.norm(T_calculated - T_1_to_2)}\n')
    print(f'Average(T[calc].P - T[orig].P) = {np.mean(points_frame_2_est - points_frame_2_, axis=0)}')

## Usage

In [39]:
check_transformation_matrix(optimize_transformation, num_points=20, point_scale=5, gaussian_scale=0.1)

Original T: 
[[ 0.07143 -0.6589   0.7488  -3.     ]
 [ 0.9446   0.2857   0.1613   5.     ]
 [-0.3202   0.6958   0.6429   7.     ]
 [ 0.       0.       0.       1.     ]]
Calculated T: 
[[ 0.067582   -0.66036386  0.74789855 -3.01267683]
 [ 0.9432136   0.28664982  0.16786893  4.99561748]
 [-0.32523956  0.69408317  0.64223655  7.03446348]
 [ 0.          0.          0.          1.        ]]
|T[Orig] - T[Calc]| = 0.03821207557344279

Average(T[calc].P - T[orig].P) = [-0.02794619  0.01017478  0.01618916]
